In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the data from a JSON file
df = pd.read_json('indeed-data-jobs-FINAL.json')
df.sample(5)

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
750,Machine Learning / Data Engineer,Alignable,United States,0,,2 days ago,2021-03-05,We're looking for engineers who are enthralled...
651,Data Platform Engineer I,Texas Capital Bank,"Richardson, TX 75082",0,,3 days ago,2021-03-05,"Overview: At Texas Capital Bank, we are drive..."
433,Data Analyst (Sales Operations),Businessolver,Remote,0,"$80,000 - $90,000 a year",7 days ago,2021-03-05,Businessolver is a technology company deliveri...
157,Data Scientist,Nomi Health,"Austin, TX 73301",0,,5 days ago,2021-03-05,WHO ARE WE? The way we pay for healthcare in A...
32,DATA SCIENTIST,Bangura Solutions,Remote,1,,16 days ago,2021-03-05,Our prestigious clients are currently looking ...


In [3]:
# these were probably removed so I'll remove them from the data
df[df.job_description == '']

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description
303,,,,0,,,2021-03-05,
463,,,,0,,,2021-03-05,
586,,,,0,,,2021-03-05,
755,,,,0,,,2021-03-05,


In [5]:
# REMOVED ROWS ABOVE
df = df[df.job_description != '']

In [6]:
# rows, columns
df.shape

(898, 8)

In [7]:
# data summary
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898 entries, 0 to 901
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        898 non-null    object
 1   company          898 non-null    object
 2   location         898 non-null    object
 3   is_remote        898 non-null    int64 
 4   salary           898 non-null    object
 5   post_date        898 non-null    object
 6   date_accessed    898 non-null    object
 7   job_description  898 non-null    object
dtypes: int64(1), object(7)
memory usage: 63.1+ KB
